In [ ]:
# pip install vertexai
# pip install -U google-cloud-aiplatform --user
# pip install transformers
# pip install sqlalchemy
# pip install mysql-connector-python
# install cloud-sql-python-connector["pg8000"] SQLAlchemy==2.0.7

In [1]:
#connect to google cloud project 
import os
key_path = 'your api-key.json file path'

os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = key_path

In [2]:
import vertexai
PROJECT_ID = "your project name"
LOCATION = "your project location"
vertexai.init(project=PROJECT_ID, location=LOCATION)

In [4]:
# word embedding을 위한 함수
import pg8000
from google.cloud.sql.connector import Connector, IPTypes
import sqlalchemy
from transformers import AutoModel, AutoTokenizer

def get_KoSimCSE():
    model = AutoModel.from_pretrained('BM-K/KoSimCSE-roberta-multitask')
    tokenizer = AutoTokenizer.from_pretrained('BM-K/KoSimCSE-roberta-multitask')

    print(type(model))
    print(type(tokenizer))

    return model, tokenizer

model, tokenizer = get_KoSimCSE()

<class 'transformers.models.roberta.modeling_roberta.RobertaModel'>
<class 'transformers.models.bert.tokenization_bert_fast.BertTokenizerFast'>


In [5]:
# VectorDB 연결
instance_connection_name = "your_instance_name"
db_user = "db_name"
db_pass = "your_db_password"
db_name = "your_vector_DB_name"

# initialize Cloud SQL Python Connector object
connector = Connector()

def getconn() -> pg8000.dbapi.Connection:
    conn: pg8000.dbapi.Connection = connector.connect(
        instance_connection_name,
        "pg8000",
        user=db_user,
        password=db_pass,
        db=db_name,
        ip_type=IPTypes.PUBLIC,
    )
    return conn

pool = sqlalchemy.create_engine(
    "postgresql+pg8000://",
    creator=getconn,
)

with pool.connect() as db_conn:
  db_conn.execute(
      sqlalchemy.text(
          "CREATE EXTENSION IF NOT EXISTS vector with schema public"
      )
  )
  db_conn.commit()

# pool.connect()하고 한 transaction만 수행 가능함.
# transaction 하나 수행하고 commit, close 하고 다시 connect 하기

In [7]:
from vertexai.preview.language_models import ChatModel, InputOutputTextPair
# TextGenerationModel, InputOutputTextPair, TextEmbeddingModel

query_text = input()

chat_model = ChatModel.from_pretrained("chat-bison@001")  #chat model 불러오기
print(type(chat_model))

chat = chat_model.start_chat(
    context="수업에 대해 궁금해하는 학생들이 과목, 교수에 대해 질문하는 서비스야",
    examples=[
        InputOutputTextPair(
            input_text="정기숙 교수님 자료구조응용 수업 어때?에서 과목명, 교수명이 뭐야?",
            output_text="과목명 자료구조응용 교수명 정기숙",
        ),
        InputOutputTextPair(
            input_text="정기숙 교수님 어때?에서 과목명, 교수명이 뭐야?",
            output_text="과목명 NULL 교수명 정기숙",
        ),
        InputOutputTextPair(
            input_text="자료구조응용 수업 어때?에서 과목명, 교수명이 뭐야?",
            output_text="과목명 자료구조응용 교수명 NULL",
        ),
    ],
    temperature=0.0,
    max_output_tokens=1024,
    top_p=0.8,
    top_k=5
)

#LLM에게 질문해서 user의 input으로부터 과목, 교수명 가져오기
key_query = chat.send_message(query_text+"에서 과목명, 교수명이 뭐야?").text
print(key_query)

<class 'vertexai.preview.language_models._PreviewChatModel'>
과목명 NULL 교수명 김정근


In [8]:
def extract(q): #LLM의 output으로부터 prof name, lecture name 추출
  lec = q.find("과목명")
  prof = q.find("교수명")
  lecture = q[lec+4:prof-1]
  professor = q[prof+4:]
  if lecture == "NULL" : lecture = None
  if professor == "NULL" : professor = None
  return lecture, professor

In [9]:
inputs = tokenizer(query_text, padding=True, truncation=True, return_tensors="pt")

embeddings, _ = model(**inputs, return_dict=False)
embedding_arr = embeddings[0][0].detach().numpy()
embedding_str = ",".join(str(x) for x in embedding_arr)
embedding_str = "["+embedding_str+"]"

lec, prof = extract(key_query)
if lec != None and prof != None :    #User의 질문 유형에 맞게 쿼리문 짜줌
  insert_stat, param = (sqlalchemy.text(
            """SELECT origin_text FROM PROFNLEC WHERE INFO LIKE :information
              ORDER BY v <-> :query_vec LIMIT 20"""
  ), {"information": f'%{lec}%{prof}%', "query_vec": embedding_str})
elif lec != None :
  insert_stat, param = sqlalchemy.text(
            """SELECT origin_text FROM PROFNLEC WHERE INFO LIKE :lecture
              ORDER BY v <-> :query_vec LIMIT 20"""
  ), {"lecture": f'%{lec}%', "query_vec": embedding_str}
elif prof != None :
  insert_stat, param = sqlalchemy.text(
            """SELECT origin_text FROM PROFNLEC WHERE INFO LIKE :professor
              ORDER BY v <-> :query_vec LIMIT 20"""
  ), {"professor": f'%{prof}%', "query_vec": embedding_str}

with pool.connect() as db_conn: # 쿼리 실행문
    result = db_conn.execute(
        insert_stat, parameters = param
    ).fetchall()

#query 결과를 문자열로 바꾸기 <- Context에는 문자열만 들어갈 수 있음
articles = ""
for res in result :
  articles += res[0]

In [10]:
chat_model = ChatModel.from_pretrained("chat-bison@001")

output_chat = chat_model.start_chat(
    context="강의를 찾는 대학생들에게 강의평들을 토대로 수업이 어떤지 알려주는 서비스야, 주어진 강의평들을 요약해서 학생들에게 알려줘" + articles + "강의평을 가져올 때는 있는 그대로 가져오지 말고 나름대로 요약해서 알려줘",
    temperature=0.3,
    max_output_tokens=1024,
    top_p=0.8,
    top_k=10
)

sentences = []
output = output_chat.send_message(query_text).text
sentences.append(output)
print(output)

김정근 교수님은 열정적인 교수님이지만 초임 교수님이라 강의력이 부족하다는 평이 있다. 과목이 어렵고 과제가 많지만 교수님이 친절하고 학생들을 많이 배려해준다고 한다.
